In [27]:
# %%
import numpy as np
import pandas as pd
import mplfinance as mpf

# Load CSV
df = pd.read_csv('auto-indig.csv')

# Parse action list from a multi-line string
Query = """
Close above EMA_21
"""
arr = np.array([line.split() for line in Query.splitlines() if line.strip()])
actlist = arr[:, arr.shape[1] // 2].tolist()  # middle word of each line

actlist

['above']

In [28]:
def above(df: pd.DataFrame, x: str, y: str, new_col: str = None) -> pd.DataFrame:
    new_col = new_col or f"{x}_above_{y}"
    df[new_col] = (df[x] > df[y]).astype(int)
    return df

def below(df: pd.DataFrame, x: str, y: str, new_col: str = None) -> pd.DataFrame:
    new_col = new_col or f"{x}_below_{y}"
    df[new_col] = (df[x] < df[y]).astype(int)
    return df

def crossed_up(df: pd.DataFrame, x: str, y: str, new_col: str = None) -> pd.DataFrame:
    new_col = new_col or f"{x}_cross_up_{y}"
    diff = df[x] - df[y]
    df[new_col] = ((diff > 0) & (diff.shift(1) <= 0)).astype(int)
    return df

def crossed_dn(df: pd.DataFrame, x: str, y: str, new_col: str = None) -> pd.DataFrame:
    new_col = new_col or f"{x}_cross_down_{y}"
    diff = df[x] - df[y]
    df[new_col] = ((diff < 0) & (diff.shift(1) >= 0)).astype(int)
    return df

In [29]:
def cabr(df: pd.DataFrame, action: str, clm1: str, clm2: str):
    return {
        "above": above,
        "below": below,
        "crossed_up": crossed_up,
        "crossed_dn": crossed_dn
    }.get(action, lambda df, x, y: "err")(df, clm1, clm2)

In [30]:
pltdf = df.iloc[50:110].copy()
pltdf['DateTime'] = pd.to_datetime(pltdf['DateTime'])
pltdf.set_index('DateTime', inplace=True)

pltdf.T

DateTime,2024-01-02 11:30:00,2024-01-02 11:45:00,2024-01-02 12:00:00,2024-01-02 12:15:00,2024-01-02 12:30:00,2024-01-02 12:45:00,2024-01-02 13:00:00,2024-01-02 13:15:00,2024-01-02 13:30:00,2024-01-02 13:45:00,...,2024-01-03 02:00:00,2024-01-03 02:15:00,2024-01-03 02:30:00,2024-01-03 02:45:00,2024-01-03 03:00:00,2024-01-03 03:15:00,2024-01-03 03:30:00,2024-01-03 03:45:00,2024-01-03 04:00:00,2024-01-03 04:15:00
Unnamed: 0,50.000000,51.000000,52.00000,53.000000,54.000000,55.000000,56.000000,57.000000,58.000000,59.00000,...,100.000000,101.000000,102.000000,103.000000,104.000000,105.000000,106.000000,107.000000,108.000000,109.000000
Open,2074.997000,2068.799000,2069.17900,2066.749000,2065.979000,2067.174000,2067.829000,2065.619000,2067.092000,2069.74200,...,2062.410000,2063.982000,2064.104000,2064.512000,2064.701000,2063.703000,2063.603000,2065.026000,2065.023000,2064.705000
High,2074.997000,2070.310000,2069.49800,2067.110000,2068.313000,2068.082000,2068.735000,2068.767000,2071.018000,2069.79500,...,2063.986000,2064.423000,2065.172000,2065.123000,2065.541000,2064.008000,2065.201000,2065.369000,2065.050000,2065.123000
Low,2068.357000,2068.247000,2066.24500,2065.016000,2065.556000,2066.635000,2064.817000,2065.358000,2066.942000,2064.82400,...,2061.598000,2062.730000,2063.365000,2063.952000,2063.575000,2063.255000,2063.380000,2064.503000,2064.352000,2064.543000
Close,2068.830000,2069.221000,2066.74000,2066.000000,2067.217000,2067.869000,2065.587000,2067.075000,2069.712000,2066.27200,...,2063.986000,2064.133000,2064.524000,2064.696000,2063.680000,2063.596000,2065.030000,2065.072000,2064.705000,2065.077000
Volume,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.00000,...,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000
EMA_21,2073.744107,2073.332916,2072.73356,2072.121418,2071.675562,2071.329511,2070.807464,2070.468149,2070.399409,2070.02419,...,2061.586551,2061.818047,2062.064043,2062.303311,2062.428465,2062.534604,2062.761459,2062.971508,2063.129098,2063.306180
RSI_14,38.055291,39.309744,34.53119,33.233525,37.400000,39.575528,34.991887,39.881391,47.427875,40.31792,...,57.755066,58.215348,59.479978,60.052734,55.098358,54.696580,60.052043,60.200442,58.167086,59.654627
SMA_10,2074.885700,2074.348200,2073.19670,2072.093500,2071.318600,2070.640900,2069.638900,2068.751500,2068.322900,2067.45230,...,2061.397100,2061.862800,2062.217400,2062.481500,2062.815000,2063.154900,2063.449900,2063.845600,2064.180100,2064.449900


In [34]:
# Suppose you have columns in your df like 'Close', 'EMA21', 'Rsi_14', etc.
# And actlist currently is something like ['above', 'above_lvl'] or similar
# But we need to know what columns to compare. You can parse from your Query too.

# Let's parse column pairs from the Query
pairs = []
for line in Query.splitlines():
    if line.strip():
        words = line.split()
        col1 = words[0]  # first column
        action = words[1]  # action: above / crossed_up / etc.
        col2 = words[-1]  # second column / level
        pairs.append((action, col1, col2))

# Now run cabr automatically for all
for action, col1, col2 in pairs:
    # If col2 is a number (like '30'), convert to float
    try:
        col2_val = float(col2)
        pltdf[col2] = col2_val  # add as a constant column for comparison
    except:
        col2_val = col2
    pltdf = cabr(pltdf, action, col1, col2_val)


pltdf.T


DateTime,2024-01-02 11:30:00,2024-01-02 11:45:00,2024-01-02 12:00:00,2024-01-02 12:15:00,2024-01-02 12:30:00,2024-01-02 12:45:00,2024-01-02 13:00:00,2024-01-02 13:15:00,2024-01-02 13:30:00,2024-01-02 13:45:00,...,2024-01-03 02:00:00,2024-01-03 02:15:00,2024-01-03 02:30:00,2024-01-03 02:45:00,2024-01-03 03:00:00,2024-01-03 03:15:00,2024-01-03 03:30:00,2024-01-03 03:45:00,2024-01-03 04:00:00,2024-01-03 04:15:00
Unnamed: 0,50.000000,51.000000,52.00000,53.000000,54.000000,55.000000,56.000000,57.000000,58.000000,59.00000,...,100.000000,101.000000,102.000000,103.000000,104.000000,105.000000,106.000000,107.000000,108.000000,109.000000
Open,2074.997000,2068.799000,2069.17900,2066.749000,2065.979000,2067.174000,2067.829000,2065.619000,2067.092000,2069.74200,...,2062.410000,2063.982000,2064.104000,2064.512000,2064.701000,2063.703000,2063.603000,2065.026000,2065.023000,2064.705000
High,2074.997000,2070.310000,2069.49800,2067.110000,2068.313000,2068.082000,2068.735000,2068.767000,2071.018000,2069.79500,...,2063.986000,2064.423000,2065.172000,2065.123000,2065.541000,2064.008000,2065.201000,2065.369000,2065.050000,2065.123000
Low,2068.357000,2068.247000,2066.24500,2065.016000,2065.556000,2066.635000,2064.817000,2065.358000,2066.942000,2064.82400,...,2061.598000,2062.730000,2063.365000,2063.952000,2063.575000,2063.255000,2063.380000,2064.503000,2064.352000,2064.543000
Close,2068.830000,2069.221000,2066.74000,2066.000000,2067.217000,2067.869000,2065.587000,2067.075000,2069.712000,2066.27200,...,2063.986000,2064.133000,2064.524000,2064.696000,2063.680000,2063.596000,2065.030000,2065.072000,2064.705000,2065.077000
Volume,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.00000,...,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000
EMA_21,2073.744107,2073.332916,2072.73356,2072.121418,2071.675562,2071.329511,2070.807464,2070.468149,2070.399409,2070.02419,...,2061.586551,2061.818047,2062.064043,2062.303311,2062.428465,2062.534604,2062.761459,2062.971508,2063.129098,2063.306180
RSI_14,38.055291,39.309744,34.53119,33.233525,37.400000,39.575528,34.991887,39.881391,47.427875,40.31792,...,57.755066,58.215348,59.479978,60.052734,55.098358,54.696580,60.052043,60.200442,58.167086,59.654627
SMA_10,2074.885700,2074.348200,2073.19670,2072.093500,2071.318600,2070.640900,2069.638900,2068.751500,2068.322900,2067.45230,...,2061.397100,2061.862800,2062.217400,2062.481500,2062.815000,2063.154900,2063.449900,2063.845600,2064.180100,2064.449900
Close_above_EMA_21,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
